Este procedimento considera uma grelha vetorial regular e encontra para cada célula os seus oito vizinhos tendo em conta a dimensão da célula

In [ ]:
grid = '/mnt/d/mystuff/fireloc/geodb_api/ref/ref_grid.shp'
neighbor = '/mnt/d/mystuff/fireloc/geodb_api/ref/ref_cells_neigh.xlsx'

In [ ]:
import geopandas as gp
from glass.gobj import create_polygon
from glass.rd.shp import shp_to_obj
from glass.gp.ext.pd import featext_to_dfcols
from glass.prop.prj import get_epsg
from glass.sql.db import create_db
from glass.wt.sql import df_to_db
from glass.sql.q import q_to_obj
from glass.wt import obj_to_tbl

In [ ]:
epsg = get_epsg(grid)

In [ ]:
grid_df = shp_to_obj(grid)
grid_df = featext_to_dfcols(grid_df, 'geometry')

In [ ]:
# Get Cellsize

# Distance between topleft and bottomleft
grid_df['height'] = ((grid_df.minx - grid_df.minx)**2 + (grid_df.maxy - grid_df.miny)**2)**0.5

# Distance between topleft and topright
grid_df['width'] = ((grid_df.maxx - grid_df.minx)**2 + (grid_df.maxy - grid_df.maxy)**2)**0.5

In [ ]:
# Get boundary including neigbords
neighbor_grid = []
neighbor_geom = []
for idx, row in grid_df.iterrows():
    # Get Polygon including 8 neigbors
    top    = row.maxy + row.height
    left   = row.minx - row.width
    bottom = row.miny - row.height
    right  = row.maxx + row.width
    neig_poly = create_polygon([
        (left, top), (left, bottom),
        (right, bottom), (right, top)
    ], api='shapely')
    
    neighbor_grid.append([row.cellid, top, left, bottom, right])
    neighbor_geom.append(neig_poly)
    
    # Apply Spatial Filter
    #exp_df = gp.GeoDataFrame(
        #[[1]], columns=['cid'], crs=f"EPSG:{str(epsg)}",
        #geometry=[neig_poly]
    #)
    
    #f_cells['tst_geom'] = f_cells.geometry.within(neig_poly)
    #f_cells = f_cells[f_cells.geometry.within(neig_poly)]
    
    #idf = gp.overlay(f_cells, exp_df, how='intersection')
neighbor_grid = gp.GeoDataFrame(
    neighbor_grid, columns=['cid', 'top', 'left', 'bottom', 'right'], crs=f"EPSG:{str(epsg)}",
    geometry=neighbor_geom
)

In [ ]:
# Create database
db = create_db('neighbors')

# Send data to database
grid_tbl = df_to_db(db, grid_df, 'grid', api='psql', epsg=epsg, geomType='Polygon', colGeom='geometry')
neig_tbl = df_to_db(db, neighbor_grid, 'cells_box', api='psql', epsg=epsg, geomType='Polygon', colGeom='geometry')

In [ ]:
neighbor_tbl = q_to_obj(db, (
    "SELECT jtbl.cellid, cells_box.cid AS isneighbor "
    "FROM cells_box INNER JOIN ("
        "SELECT cellid, ST_Centroid(geom) AS geom "
        "FROM grid"
    ") AS jtbl "
    "ON ST_ContainsProperly(cells_box.geom, jtbl.geom) "
    "WHERE cells_box.cid <> jtbl.cellid"
))

obj_to_tbl(neighbor_tbl, neighbor)

In [ ]:
grid_df.head()

In [ ]:
#neighbor_grid.to_file('/mnt/d/mystuff/fireloc/tstdata/teste.shp')